In [1]:
import pandas as pd

In [21]:
uncleaned_data = pd.read_csv("D:/Projects/Machine Learning Applications for Soccer/player_database.csv")
uncleaned_data.head()


,ID,Name,Age,Career_Phase,Position,Sub_Position,Preferred_Foot,Height,Nationality,Birth_Date,...,Minutes_Per_Game,Recent_Goals,Recent_Assists,Recent_Minutes,Recent_Yellows,Recent_Reds,Recent_Games,transfermarkt_url,fbref_url,injury_history
0,10,Miroslav Klose,46.477755,twilight,Attack,Centre-Forward,right,184.0,Germany,1978-06-09,...,64.764706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26,Roman Weidenfeller,44.317591,twilight,Goalkeeper,Goalkeeper,left,190.0,Germany,1980-08-06,...,88.868421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,65,Dimitar Berbatov,43.832991,twilight,Attack,Centre-Forward,NaN,NaN,Bulgaria,1981-01-30,...,72.032787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,77,Lúcio,46.565366,twilight,Defender,Centre-Back,NaN,NaN,Brazil,1978-05-08,...,76.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,80,Tom Starke,43.704312,twilight,Goalkeeper,Goalkeeper,right,194.0,Germany,1981-03-18,...,90.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
uncleaned_data.isnull().sum()


ID                       0
Name                     0
Age                     47
Career_Phase             0
Position                 0
Sub_Position           184
Preferred_Foot        2598
Height                2339
Nationality            380
Birth_Date              47
Contract_Expiry      12110
Current_Value         1541
Peak_Value            1541
Current_Club             0
League_ID                0
Squad_Size               0
Team_Average_Age      1858
Team_Total_Value     32417
Coach                32417
Career_Games          6967
Career_Minutes        6967
Career_Goals          6967
Career_Assists        6967
Career_Yellows        6967
Career_Reds           6967
Goals_Per_Game        6967
Assists_Per_Game      6967
Minutes_Per_Game      6967
Recent_Goals         24567
Recent_Assists       24567
Recent_Minutes       24567
Recent_Yellows       24567
Recent_Reds          24567
Recent_Games         24567
transfermarkt_url    32417
fbref_url            32417
injury_history       32417
d

In [23]:
uncleaned_data.isna().sum()


ID                       0
Name                     0
Age                     47
Career_Phase             0
Position                 0
Sub_Position           184
Preferred_Foot        2598
Height                2339
Nationality            380
Birth_Date              47
Contract_Expiry      12110
Current_Value         1541
Peak_Value            1541
Current_Club             0
League_ID                0
Squad_Size               0
Team_Average_Age      1858
Team_Total_Value     32417
Coach                32417
Career_Games          6967
Career_Minutes        6967
Career_Goals          6967
Career_Assists        6967
Career_Yellows        6967
Career_Reds           6967
Goals_Per_Game        6967
Assists_Per_Game      6967
Minutes_Per_Game      6967
Recent_Goals         24567
Recent_Assists       24567
Recent_Minutes       24567
Recent_Yellows       24567
Recent_Reds          24567
Recent_Games         24567
transfermarkt_url    32417
fbref_url            32417
injury_history       32417
d

In [24]:
cleaned_data = uncleaned_data.copy()

In [25]:
columns_to_drop = [
    'transfermarkt_url', 'fbref_url', 'injury_history', 
    'Team_Total_Value', 'Coach',
    'Recent_Goals', 'Recent_Assists', 'Recent_Minutes',
    'Recent_Yellows', 'Recent_Reds', 'Recent_Games'
]
cleaned_data = cleaned_data.drop(columns=columns_to_drop)


In [26]:
cleaned_data['Birth_Date'] = pd.to_datetime(cleaned_data['Birth_Date'], errors='coerce')
cleaned_data['Age'] = cleaned_data['Age'].fillna(
    (pd.Timestamp.now() - cleaned_data['Birth_Date']).dt.total_seconds() / (365.25 * 24 * 60 * 60)
)
age_by_position = cleaned_data.groupby('Position')['Age'].transform('median')
cleaned_data['Age'] = cleaned_data['Age'].fillna(age_by_position)
cleaned_data['Age'] = cleaned_data['Age'].fillna(cleaned_data['Age'].median())


In [27]:
cleaned_data['Sub_Position'] = cleaned_data['Sub_Position'].fillna(cleaned_data['Position'] + '_General')


In [28]:
cleaned_data['Nationality'] = cleaned_data['Nationality'].fillna('Unknown')


In [29]:
cleaned_data['Preferred_Foot'] = cleaned_data['Preferred_Foot'].fillna('Unknown')


In [30]:
height_by_position = cleaned_data.groupby('Position')['Height'].transform('median')
cleaned_data['Height'] = cleaned_data['Height'].fillna(height_by_position)


In [31]:
cleaned_data['Has_Contract_Info'] = cleaned_data['Contract_Expiry'].notna().astype(int)
cleaned_data['Contract_Expiry'] = pd.to_datetime(cleaned_data['Contract_Expiry'], errors='coerce')
cleaned_data['Contract_Months_Remaining'] = (cleaned_data['Contract_Expiry'] - pd.Timestamp.now()).dt.total_seconds() / (30 * 24 * 60 * 60)
cleaned_data['Contract_Months_Remaining'] = cleaned_data['Contract_Months_Remaining'].fillna(-1)


In [32]:
cleaned_data['Current_Value'] = cleaned_data['Current_Value'].fillna(0)
cleaned_data['Peak_Value'] = cleaned_data['Peak_Value'].fillna(0)


In [33]:
cleaned_data['Team_Average_Age'] = cleaned_data['Team_Average_Age'].fillna(cleaned_data['Team_Average_Age'].mean())


In [34]:
career_stats = ['Career_Games', 'Career_Minutes', 'Career_Goals', 'Career_Assists', 
                'Career_Yellows', 'Career_Reds', 'Goals_Per_Game', 'Assists_Per_Game', 
                'Minutes_Per_Game']

for col in career_stats:
    # Fill with 0 for new players, median for others based on position
    position_medians = cleaned_data.groupby('Position')[col].transform('median')
    cleaned_data[col] = cleaned_data[col].fillna(position_medians)
    cleaned_data[col] = cleaned_data[col].fillna(0)  # Fill remaining NAs with 0


In [35]:
categorical_cols = ['Position', 'Sub_Position', 'Preferred_Foot', 'Career_Phase', 'Nationality']
cleaned_data = pd.get_dummies(cleaned_data, columns=categorical_cols, drop_first=True)


In [36]:
from sklearn.preprocessing import RobustScaler  # RobustScaler is better for data with outliers
numerical_cols = ['Age', 'Height', 'Current_Value', 'Peak_Value', 'Team_Average_Age'] + career_stats
scaler = RobustScaler()
cleaned_data[numerical_cols] = scaler.fit_transform(cleaned_data[numerical_cols])


In [37]:
cleaned_data = cleaned_data.drop(columns=['Birth_Date', 'Contract_Expiry'])  # Remove date columns after extracting features
cleaned_data = cleaned_data.reset_index(drop=True)


In [38]:
print("Original shape:", uncleaned_data.shape)
print("Cleaned shape:", cleaned_data.shape)
print("\nRemaining missing values:")
print(cleaned_data.isnull().sum())

Original shape: (32417, 37)
Cleaned shape: (32417, 228)

Remaining missing values:
ID                       0
Name                     0
Age                      0
Height                   0
Current_Value            0
                        ..
Nationality_Venezuela    0
Nationality_Vietnam      0
Nationality_Wales        0
Nationality_Zambia       0
Nationality_Zimbabwe     0
Length: 228, dtype: int64


In [39]:
cleaned_data.to_csv("D:/Projects/Machine Learning Applications for Soccer/cleaned_player_database.csv", index=False)
print("Dataset saved successfully!")

Dataset saved successfully!


In [41]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [42]:
cleaned_data.columns

Index(['ID', 'Name', 'Age', 'Height', 'Current_Value', 'Peak_Value',
       'Current_Club', 'League_ID', 'Squad_Size', 'Team_Average_Age',
       ...
       'Nationality_United Arab Emirates', 'Nationality_United States',
       'Nationality_Unknown', 'Nationality_Uruguay', 'Nationality_Uzbekistan',
       'Nationality_Venezuela', 'Nationality_Vietnam', 'Nationality_Wales',
       'Nationality_Zambia', 'Nationality_Zimbabwe'],
      dtype='object', length=228)

In [ ]:
# Add performance metrics calculation
def calculate_performance_metrics(df):
    """Calculate additional performance metrics"""
    df['value_efficiency'] = df['Current_Value'] / df['Peak_Value']
    df['goals_per_minute'] = df['Career_Goals'] / df['Career_Minutes']
    df['assists_per_minute'] = df['Career_Assists'] / df['Career_Minutes']
    
    # Calculate progression rate (how quickly player is developing)
    df['value_progression'] = (df['Current_Value'] - df['Peak_Value']) / df['Age']
    
    return df

# Add career phase specific features
def add_career_phase_features(df):
    """Add features specific to career phases"""
    # Calculate time in current phase
    phase_age_ranges = {
        'breakthrough': (0, 21),
        'development': (22, 25),
        'peak': (26, 29),
        'twilight': (30, 100)
    }
    
    for phase, (min_age, max_age) in phase_age_ranges.items():
        df[f'is_{phase}_age'] = (df['Age'] >= min_age) & (df['Age'] <= max_age)
    
    return df

In [ ]:
cleaned_data = calculate_performance_metrics(cleaned_data)
cleaned_data = add_career_phase_features(cleaned_data)